# GraphGuard

***Locate and find Classes in Apks with updated Obfuscation Mapping***


Processing Steps:
* Using Strings as characteristic to locate and find Classes 

In [ ]:
%matplotlib notebook

In [ ]:
from androguard.misc import AnalyzeAPK
from androguard.session import Save, Session, Load
from androguard.core.bytecode import FormatClassToJava
from androguard.core.analysis.analysis import StringAnalysis

import networkx as nx
import matplotlib.pyplot as plt

import unittest
import multiprocessing

from os import path
from collections import defaultdict, Counter

# Loading Androguard

The following code loads the files and starts Androguard

It should support multiprocessing, however the Pipe communication seems to break when transmitting the processed Androguard Objects. I suspect the Object is simply too big for Pickle to serialize or another component in the transmitting chain.

In [ ]:
AG_SESSION_FILE = "./Androguard.ag"
MAX_USAGE_COUNT_STR = 20
MULTIPROCESS_FILES = True

file_paths = (
    "../../../Downloads/com.snapchat.android_10.85.5.74-2067_minAPI19(arm64-v8a)(nodpi)_apkmirror.com.apk",
    "../../../Downloads/com.snapchat.android_10.86.5.61-2069_minAPI19(arm64-v8a)(nodpi)_apkmirror.com.apk"
)

In [ ]:
def load_androguard(file_path, force_reload=False, write_session=True):
    if (not force_reload) and path.exists(AG_SESSION_FILE):
        print("Loading Existing Session")
        s = Load(AG_SESSION_FILE)
    else:
        print("Loading Session from Apk")
        s = Session()
        a, d, dx = AnalyzeAPK(file_path, s)
        if write_session:
            print("Saving Loaded Session to", AG_SESSION_FILE)
            Save(s, AG_SESSION_FILE)
    return a, d, dx

In [ ]:
# Multiprocessing not working
"""
def multiprocess_files(file_paths):
    parent_conn, child_conn = multiprocessing.Pipe(False)


    def post_result(file_path, conn):
        value = load_androguard(file_path, True, False)
        conn.send((file_path, value))

    ps =  [multiprocessing.Process(target=post_result, args=(f, child_conn)) for f in file_paths]

    def apply_map(f, i):
        for x in i:
            f(x)
    
    assert len(file_paths) == 2
    print("Starting multiprocessing Files")
    
    # Serialization with Pickle requires higher recursion limit
    import sys
    previous_recursion = sys.getrecursionlimit()
    sys.setrecursionlimit(50000)
    
    
    apply_map(multiprocessing.Process.start, ps)

    values = (q.get(), q.get())
    r = tuple(map(lambda x: x[1], sorted(values, key=lambda x: file_paths.index(x[0]))))
    
    
    apply_map(multiprocessing.Process.join, ps)
    
    print("Finished all processes")
    sys.setrecursionlimit(previous_recursion)
    return r

if MULTIPROCESS_FILES:
    (a, d, dx), (a2, d2, dx2) = multiprocess_files(file_paths)
else:
    (a, d, dx), (a2, d2, dx2) = tuple(map(lambda x: load_androguard(x, True, False), file_paths))
"""
a, d, dx = load_androguard(file_paths[0], True, False)

### Utility Functions to work with Androguard and Java Representations

* Converting Parameter types to TypeDescriptor Format
* Strip return type (not used for hooking)
* Method Representation Format

Loaded with Unit Tests

In [ ]:
# https://source.android.com/devices/tech/dalvik/dex-format#typedescriptor
type_descriptors = {
    "void": "V",
    "boolean": "Z",
    "byte": "B",
    "short": "S",
    "char": "C",
    "int": "I",
    "long": "J",
    "float": "F",
    "double": "D"
}

def get_as_type_descriptor(arg):
    if arg.endswith("[]"):
        return "[" + get_as_type_descriptor(arg[:-2])
    if arg in type_descriptors:
        return type_descriptors[arg]
    return FormatClassToJava(arg)

In [ ]:
def strip_return(name):
    return name[1:name.index(")")]

In [ ]:
def get_method_repr(class_name, method_name, param_types):
    return f"{class_name}#{method_name}({param_types})"

In [ ]:
tests_1 = (
    ("java.lang.String", "Ljava/lang/String;"),
    ("java.lang.String[]", "[Ljava/lang/String;"),
    ("void", "V"),
    ("int[]", "[I"),
    ("char", "C"), 
    ("java.lang.Object[][]", "[[Ljava/lang/Object;")
)

tests_2 = (
    ("(I)I", "I"), 
    ("(C)Z", "C"),
    ("(Ljava/lang/CharSequence; I)I", "Ljava/lang/CharSequence; I")
)

class TestFunction(unittest.TestCase):
    def test_add(self):
        for test, val in tests_1:
            self.assertEqual(get_as_type_descriptor(test), val)
        for test, val in tests_2:
            self.assertEqual(strip_return(test), val)


unittest.main(argv=[''], verbosity=2, exit=False)

# Method Declarations

Lightweight Method Declaration for internal representation of a Method / Hook.

Not keeping Androguard Objects in memory to avoid high memory usage.

In [ ]:
class MethodDec:
    def __init__(self, class_name, name, *param_types):
        self.name = name
        self.class_name = class_name
        self.param_types = param_types
    
    def get_method_parameter_types(self):
        return list(map(get_as_type_descriptor, self.param_types))
    
    
    def param_types_repr(self):
        return " ".join(method_dec.get_method_parameter_types())
    
    def get_formatted_class(self):
        return FormatClassToJava(class_name)
    
    def __repr__(self):
        return f'MethodDec({get_method_repr(self.class_name, self.name, self.param_types_repr())})'

### List of Methods

Defining the list of methods to find (obviously requires full class names)

In [ ]:
decs_to_find = [
    MethodDec("rD5", "a", "rD5", "qD5"),
    MethodDec("MSg", "j0", "SGd")
]

# Processing

## Strings as Characteristics

Extracting Strings used either in the given methods directly or in the classes the methods define

In [ ]:
# Key:   TypeDescriptor Representation of class
# Value: Androguard Class Analysis Object

resolved_classes = { i: dx.get_class_analysis(i)
                    for i in map(lambda x: FormatClassToJava(x.class_name), decs_to_find) }

In [ ]:
resolved_methods = []

# Loop through resolved classes
for method_dec, (class_name, class_analysis) in zip(decs_to_find, resolved_classes.items()):
    # Loop through all methods in the resolved class
    for method in class_analysis.get_methods():
        
        # Check Method Name
        if method_dec.name != method.name:
            continue
        
        # Check Parameter Types
        m_dec_types = " ".join(method_dec.get_method_parameter_types())
        if m_dec_types != strip_return(str(method.get_descriptor())):
            continue
        
        # Matching Method Declaration found
        print("Found Class and Method", method_dec)
        resolved_methods.append(method)
        break
    else:
        raise Exception(f"One method was not resolved: {method_dec}")

### Utility functions for working with dx.get_strings()

Filters Strings and xrefs to Strings. Only allow strings with (#xrefs < MAX_USAGE_COUNT_STR) to be used as characteristic to locate classes

In [ ]:
def get_filtered_strs(dx):
    """
    Loops through all strings that are referenced less than MAX_USAGE_COUNT_STR times and hence can be 
    used as characteristic for finding methods or classes.
    """
    return ((s, xrefs) 
            for s, xrefs in map(lambda s: (s, s.get_xref_from()), dx.get_strings()) 
            if len(xrefs) <= MAX_USAGE_COUNT_STR)


def get_xrefs_if_usable(s):
    """
    Loops through xrefs of a string only if the number of references does not exceed MAX_USAGE_COUNT_STR.
    """
    xrefs = s.get_xref_from()
    if len(xrefs) > MAX_USAGE_COUNT_STR:
        return
    yield from xrefs

Building Maps of MethodDec and ClassNames associated to lists containing strings used in them

In [ ]:
m_strs, c_strs = defaultdict(list), defaultdict(list)

for s, xrefs in get_filtered_strs(dx):
    for x in xrefs:
        c_ref, m_ref = x

        if c_ref.name not in resolved_classes:
            # XReference not in a Class or method that we need to find
            continue

        # Loop through each method and find methods in this class
        for r_m, m_dec in zip(resolved_methods, decs_to_find):
            if r_m.class_name != c_ref.name:
                continue

            if m_ref == r_m:
                # String is used in this method
                m_strs[m_dec].append(s.value)
            else:
                # String is used in another method in the same class
                c_strs[c_ref.name].append(s.value)

### Count occurrences of strings

Converting list of strings to a Counter object for faster comparisons

In [ ]:
m_strs = {k: Counter(v) for k, v in m_strs.items()}
c_strs = {k: Counter(v) for k, v in c_strs.items()}

In [ ]:
def flat_map(f, li):
    """
    Maps values with function f recursively on all Iterables (except Strings)
    Flattened by using Subgenerator Delegation
    """
    from collections.abc import Iterable
    for i in li:
        # str will cause a recursion depth error (Iterator of str returns Iterable str)
        if isinstance(i, Iterable) and not isinstance(i, str):
            yield from flat_map(f, i)
        else:
            yield f(i)

# def flat_map(f, li): return (f(y) for x in li for y in x)

### Searching for Found Strings

Tries to resolve Classes and methods with the strings previously found

In [ ]:
for s in dx.get_strings():
    for m_dec, m_set in m_strs.items():
        c_name = m_dec.class_name
        c_set = c_strs[FormatClassToJava(c_name)]
        
        if s.value in m_set:
            for x in get_xrefs_if_usable(s):
                m_xref = x[1]
                print("Possible MCandidate:", get_method_repr(c_name, m_ref.name, strip_return(str(m_xref.descriptor))))
        elif s.value in c_set:
            for x in get_xrefs_if_usable(s):
                c_xref = x[0]
                print("Possible CCandidate:", c_name)